# 训练过程中奖回调函数作用于模型

## ModelCheckpoint与EarlyStopping回调函数

In [1]:
import keras

Using TensorFlow backend.


In [2]:
callback_list = [
    keras.callbacks.EarlyStopping(monitor='acc, patience=1'),
    keras.callbacks.ModelCheckpoint(filepath='my_model.h5', monitor='val_loss', save_best_only=True)
]

In [3]:
callback_list

In [4]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

In [5]:
model.fit(x, y, epochs=10, batch_size=32, callbacks=callback_list, validation_data=(x_val, y_val))

## ReduceLROnPlateau回调函数

In [6]:
# 如果验证损失不再改善，可以使用这个回调函数来降低学习率

In [7]:
callback_list = [
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)
]

In [8]:
model.fit(x, y, epochs=10, batch_size=32, callbacks=callback_list, validation_data=(x_val, y_val))

## 编写自己的回调函数

In [9]:
import keras
import numpy as np

In [10]:
class ActivationLogger(keras.callbacks.Callback):
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input, layer_outputs)

    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
        validation_sample = self.validation_data[0][0:1]
        activations = self.activations_model.predict(validation_sample)
        f = open('activations_at_epoch' + str(epoch) + '.npz', 'w')
        np.savez(f, activations)
        f.close()

# TensorBoard简介：TensorFlow的可视化框架

## 使用了TensorBoard的文本分类模型

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

In [2]:
max_features = 2000
max_len = 500

In [3]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

In [4]:
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

In [5]:
model = keras.models.Sequential()
model.add(layers.Embedding(input_dim=max_features, output_dim=128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(layers.MaxPooling1D(pool_size=5))
model.add(layers.Conv1D(filters=32, kernel_size=7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(units=1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

## 为TensorBoard日志文件创建一个目录

In [7]:
%mkdir my_log_dir

## 使用一个TensorBoard回调函数来训练模型

In [12]:
callbacks = [keras.callbacks.TensorBoard(log_dir='../my_log_dir/', histogram_freq=1, embeddings_freq=1)]

In [13]:
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2, callbacks=callbacks)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 114s - loss: 0.0966 - acc: 0.1602 - val_loss: 1.1935 - val_acc: 0.2400
Epoch 2/20
20000/20000 [==============================] - 120s - loss: 0.0952 - acc: 0.1424 - val_loss: 1.2227 - val_acc: 0.2312
Epoch 3/20
20000/20000 [==============================] - 109s - loss: 0.0937 - acc: 0.1286 - val_loss: 1.5181 - val_acc: 0.2114
Epoch 4/20
20000/20000 [==============================] - 115s - loss: 0.0965 - acc: 0.1229 - val_loss: 1.3004 - val_acc: 0.2096
Epoch 5/20
20000/20000 [==============================] - 119s - loss: 0.0910 - acc: 0.1015 - val_loss: 1.3312 - val_acc: 0.1912
Epoch 6/20
20000/20000 [==============================] - 106s - loss: 0.0926 - acc: 0.0868 - val_loss: 1.3274 - val_acc: 0.1966
Epoch 7/20
20000/20000 [==============================] - 108s - loss: 0.0954 - acc: 0.0897 - val_loss: 1.3287 - val_acc: 0.1944
Epoch 8/20
20000/20000 [========================

In [14]:
model.save('imdb_model.h5')

In [15]:
# TensorBoard可视化如下

<figure class='half'>
    <img src='scalars.png' height=500 width=500>
    <img src='projector.png' height=500 width=500>
    <img src='histograms.png' height=500 width=500>
    <img src='graph.png' height=500 width=500>
    <img src='distributions.png' height=500 width=500>
</figure>

In [16]:
# 也可以使用plot_model函数将模型绘制为层组成的图

In [2]:
from keras.utils import plot_model
from keras.models import load_model

In [3]:
model = load_model(filepath='./imdb_model.h5')

In [4]:
plot_model(model=model, to_file='./model.png')

<img src='model.png' height=200 width=200>

In [5]:
# 可视化模型的拓扑结构

In [6]:
plot_model(model=model, show_shapes=True, to_file='./model_shapes.png')

<img src='model_shapes.png' height=300 width=300>